In [1]:
%pip install huggingsound SpeechRecognition PyAudio

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/14.6 MB 3.2 MB/s eta 0:00:05
     - -------------------------------------- 0.5/14.6 MB 5.6 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/14.6 MB 12.1 MB/s eta 0:00:02
     ---------- ----------------------------- 3.7/14.6 MB 21.7 MB/s eta 0:00:01
     ----------------- ---------------------- 6.6/14.6 MB 30.0 MB/s eta 0:00:01
     ----------------------- ---------------- 8.5/14.6 MB 32.0 MB/s eta 0:00:01
     ------------------------------- ------- 11.7/14.6 MB 54.4 MB/s eta 0:00:01
     --------------------------------------  14.6/14.6 MB 59.5 MB/s eta 0:00:01
     --------------------------------------- 14.6/14.6 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1
Note: you may need to restart the kernel to use updated pac

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0 requires torch==1.11.0, but you have torch 1.12.1 which is incompatible.


In [12]:
#S2T
import torch
from transformers import pipeline, Wav2Vec2ForCTC, Wav2Vec2Processor
from huggingsound import SpeechRecognitionModel
import speech_recognition as sr
import io
import soundfile as sf
#T2S
import torchaudio
from speechbrain.pretrained import Tacotron2
from speechbrain.pretrained import HIFIGAN

In [17]:
tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder")

In [3]:
mel_output, mel_length, alignment = tacotron2.encode_text("Mary had a little lamb")

In [4]:
waveforms = hifi_gan.decode_batch(mel_output)

In [ ]:
torchaudio.save('example_TTS.wav',waveforms.squeeze(1), 22050)

In [3]:
LANG_ID = "en"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
SAMPLES = 10

In [6]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
r = sr.Recognizer()

In [18]:
with sr.Microphone(sample_rate=16000) as mic:
  print('listening')
  while True: 
    r.adjust_for_ambient_noise(mic)
    audio = r.listen(mic)
    # data, samplerate = sf.read(io.BytesIO(audio.get_wav_data()))
    try:
      # inputs = processor(data, sampling_rate=16_000, return_tensors="pt", padding=True)
      # with torch.no_grad():
      #     logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

      # predicted_ids = torch.argmax(logits, dim=-1)
      # text_model = processor.batch_decode(predicted_ids)[0]
      # print('model ---> ', text_model)
      text_google = r.recognize_google(audio)
      print('google --> ', text_google)
      # TTS
      mel_output, mel_length, alignment = tacotron2.encode_text(text_google)
      waveforms = hifi_gan.decode_batch(mel_output)
      torchaudio.save('hello.wav',waveforms.squeeze(1), 22050)
    except Exception as e:
      print(e)

listening
google -->  hi penguins live in Antarctica


KeyboardInterrupt: 